# 作业

In [9]:
class Data:
    
    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, **kwargs):
        r"""
        Args:
            x (Tensor, optional): 节点属性矩阵，大小为`[num_nodes, num_node_features]`
            edge_index (LongTensor, optional): 边索引矩阵，大小为`[2, num_edges]`，第0行为 尾节点，第1行为头节点，头指向尾
            edge_attr (Tensor, optional): 边属性矩阵，大小为`[num_edges, num_edge_features]`
            y (Tensor, optional): 节点或图的标签，任意大小(，其实也可以是边的标签)
        """
        self.x = x
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.y = y
        for key, item in kwargs.items():
            if key == 'num_nodes':
                self.__num_nodes__ = item
            else:
                self[key] = item
                
class MyData(Data):
    def __init__(self, node_paper=None, node_institute=None, node_author=None,
                 edge_index_author_institute=None,edge_index_paper_author=None,y=None):  
        super(MyData,self).__init__(y=y)  
        self.node_paper = node_paper
        self.node_institute = node_institute
        self.node_author = node_author
        self.edge_index_author_institute = edge_index_author_institute
        self.edge_index_paper_author = edge_index_paper_author
    
    @property
    def node_paper_num(self):
        return self.node_paper.shape[0]
    
    @property
    def node_author_num(self):
        return self.node_author.shape[0]
    
    @property
    def node_institute_num(self):
        return self.node_institute.shape[0]

# 学习笔记

## 简单图论

In [7]:
history

# 学习笔记
# 学习笔记
## 简单图论
使用`nvidia-smi`查看显卡驱动是否正确安装
%history
history
history


## pyG环境配置

环境：阿里云gpu计算型（阿里云gpu虚拟型不能安装显卡驱动，计算型会自动装）



使用`nvidia-smi`查看显卡驱动是否正确安装

安装1.8.1版本的pytorch和11.1版本的cudatoolkit

`conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia`

确认是否正确安装，正确的安装应出现下方的结果

```
$ python -c "import torch; print(torch.__version__)"
# 1.8.1
$ python -c "import torch; print(torch.version.cuda)"
# 11.1
```

安装对应版本的PyG
```
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-geometric
```

注：在阿里云上安装`pip install torch-geometric`这一步会出错，使用`pip install pytest-runner`之后再安装torch-geometric就成功了

In [6]:
history

# 学习笔记
# 学习笔记
## 简单图论
使用`nvidia-smi`查看显卡驱动是否正确安装
%history
history
